In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

In [ ]:
C = 100
G_L, E_L = 10, -70
G_Na, E_Na = 3.6, 55
G_K, E_K = 1.6, -90
G_T, E_Ca = 0.22, 120

In [ ]:
def alpham(V):
    if V == (-0.035):
        return 10**3 / np.exp(-100*(V+0.035))
    else:
        return (10**5 * (V+0.035)) / (1 - np.exp(-100*(V+0.035)))
def alphan(V):
    if V == (-0.034):
        return (5 * 10**2) / np.exp(-100*(V+0.034))
    else:
        return (5 * 10**4 * (V+0.034)) / (1 - np.exp(-100*(V+0.034)))

def alphah(V): return 350*np.exp(-50*(V+0.058))

def betam(V): return 4000*np.exp(-(V+0.06)/0.018)
def betan(V): return 625*np.exp(-12.5*(V+0.044))
def betah(V): return 5000 / (1 + np.exp(-100*(V+0.028)))

def m_i(V): return alpham(V) / (alpham(V) + betam(V))
def m_ti(V): return 1 / (1+np.exp(-(V+0.052) / 0.0074))
def h_ti(V): return 1 / (1+np.exp(500*(V+0.076)))

def t_ht(V):
    if V>=(-0.080):
        return 0.028 + 0.001*np.exp(-(V+0.022) / 0.0105)
    else:
        return 0.001*np.exp(15*(V+0.467))

In [ ]:
t = np.arange(0, 750, 0.01) #intervalo de tempo

def J_inj(t):   #corrente injetada
    J_base = -50
    ti_base = 0
    tf_base = 250
    J_degrau = 15
    ti_degrau = 250
    tf_degrau = 500
    return J_base*(t>ti_base)-J_base*(t>tf_base)+J_degrau*(t>ti_degrau)-J_degrau*(t>tf_degrau)+J_base*(t>500)


def dALLdt(X,t):
    V, h, n, h_t = X
    dVdt = (J_inj(t) - G_L*(V-E_L) - G_Na * m_i(V)**3 * h*(V-E_Na) - G_K * n**4 * (V-E_K) - G_T * m_ti(V)**2 * h_t*(V-E_Ca)) / C
    dhdt = alphah(V)*(1-h) - betah(V) * h
    dndt = alphan(V)*(1-n) - betan(V) * n
    dh_tdt = (h_ti(V) - h_t) / t_ht(V)

    return dVdt, dhdt, dndt, dh_tdt

In [ ]:
v0, h0, n0, h_t0 = -70, 0, 0, 0
X = odeint(dALLdt, [v0, h0, n0, h_t0], t)
V = X[:,0]
h = X[:,1]
n = X[:,2]
h_t = X[:,3]

In [ ]:
plt.subplot(2,1,1)
plt.plot(t, V, 'k')

plt.subplot(2,1,2)
plt.plot(t, n, 'r', label='n')
plt.plot(t, h, 'g', label='h')
plt.plot(t, h_t, 'b', label='ht')
plt.legend()
plt.show()